In [1]:
import json
import os
import torch
import clip
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity



In [2]:
# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)



In [3]:
# Load product data
with open("img.json", "r") as f:
    products = json.load(f)



In [4]:
# Step 1: Encode all product images
product_features = []
product_ids = []

for product in products:
    image_path = os.path.join("products", os.path.basename(product["image_path"]))
    try:
        image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
        with torch.no_grad():
            feature = model.encode_image(image).cpu().numpy()
        product_features.append(feature)
        product_ids.append(product["id"])
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")



In [5]:
# Stack features
product_features = torch.tensor([f[0] for f in product_features])



/var/folders/l9/bpfz3cfd4m9fwlfld0g3dswc0000gn/T/ipykernel_8272/2165487848.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  product_features = torch.tensor([f[0] for f in product_features])


In [6]:
# Step 2: Load and encode user-uploaded image
user_image_path = "wall.jpg"  # <- replace this with actual test image
user_image = preprocess(Image.open(user_image_path)).unsqueeze(0).to(device)



In [7]:
with torch.no_grad():
    user_feature = model.encode_image(user_image).cpu().numpy()



In [8]:
# Step 3: Compute cosine similarity
similarities = cosine_similarity(user_feature, product_features)[0]
top_indices = similarities.argsort()[-5:][::-1]  # top 5



In [9]:
# Step 4: Show recommendations
print("Top Recommendations:")
for idx in top_indices:
    product_id = product_ids[idx]
    matched_product = next((p for p in products if p["id"] == product_id), None)
    if matched_product:
        print(f"\nName: {matched_product['name']}")
        print(f"Category: {matched_product['category']}")
        print(f"Image: {matched_product['image_path']}")
        print(f"Tags: {', '.join(matched_product['tags'])}")

Top Recommendations:

Name: Al Kaaba Premium Acrylic Horizontal Wall Art
Category: acrylic
Image: products/acrylic004.jpg
Tags: kaaba, horizontal, religious, islamic

Name: Arabic Calligraphy Premium Acrylic Vertical Wall Art
Category: acrylic
Image: products/acrylic006.jpg
Tags: calligraphy, arabic, vertical, islamic

Name: Allah Is For All Premium Acrylic Horizontal Wall Art
Category: acrylic
Image: products/acrylic005.jpg
Tags: allah, horizontal, spiritual, islamic

Name: 3D Jesus Artistry Premium Acrylic Square Wall Art
Category: acrylic
Image: products/acrylic001.jpg
Tags: Jesus, 3D, square, spiritual

Name: 7 White Running Horses in The Sea With Sunrise Premium Acrylic Horizontal Wall Art
Category: acrylic
Image: products/acrylic003.jpg
Tags: horses, sunrise, horizontal, nature


In [10]:
import pickle
import numpy as np

# After you computed product_features and product_ids
data_to_save = {
    "product_features": np.array([f[0] for f in product_features]),  # NumPy array
    "product_ids": product_ids,
    "products": products
}

with open("product_features.pkl", "wb") as f:
    pickle.dump(data_to_save, f)

print("✅ Product features saved to product_features.pkl")


✅ Product features saved to product_features.pkl


In [11]:
import pickle

with open("product_features.pkl", "rb") as f:
    saved_data = pickle.load(f)

product_features = saved_data["product_features"]
product_ids = saved_data["product_ids"]
products = saved_data["products"]

print("✅ Product features loaded from product_features.pkl")


✅ Product features loaded from product_features.pkl


array([ 0.49003962, -0.1179328 ,  0.07294275,  0.27493745, -0.14781323,
        0.03469466,  0.32410622,  0.52520555], dtype=float32)

In [21]:
import json
import os
import torch
import clip
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import numpy as np

print("Lib are imported")

# ===== Step 1: Load CLIP Model =====
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

print("Clip model is loaded")

# ===== Step 2: Load saved features =====
with open("product_features.pkl", "rb") as f:
    saved_data = pickle.load(f)
    print("Pkl model is loaded")

# Flatten features to shape (N, 512)
product_features = np.array([np.array(f).flatten() for f in saved_data["product_features"]])
product_ids = saved_data["product_ids"]
products = saved_data["products"]
print("Flatten")
# ===== Step 3: Load and encode test image =====
test_image_path = "wall.jpg"
image = preprocess(Image.open(test_image_path)).unsqueeze(0).to(device)

with torch.no_grad():
    test_feature = model.encode_image(image).cpu().numpy().flatten().reshape(512, 1)
    # test_feature = np.array(test_feature).reshape(1, -1)

# ===== Step 4: Compute cosine similarity =====
similarities = cosine_similarity(test_feature, product_features)[0]
top_indices = similarities.argsort()[-5:][::-1]  # Top 5 matches

# ===== Step 5: Print recommendations =====
print("🔍 Top Recommendations for wall.jpg:\n")
for idx in top_indices:
    product_id = product_ids[idx]
    matched_product = next((p for p in products if p["id"] == product_id), None)
    if matched_product:
        print(f"Name: {matched_product['name']}")
        print(f"Category: {matched_product['category']}")
        print(f"Image: {matched_product['image_path']}")
        print(f"Tags: {', '.join(matched_product['tags'])}")
        print("-" * 40)


Lib are imported
Clip model is loaded
Pkl model is loaded
Flatten
🔍 Top Recommendations for wall.jpg:

Name: Allah Is For All Premium Acrylic Horizontal Wall Art
Category: acrylic
Image: products/acrylic005.jpg
Tags: allah, horizontal, spiritual, islamic
----------------------------------------
Name: 7 Running White Horses in The Sea Premium Acrylic Horizontal Wall Art
Category: acrylic
Image: products/acrylic002.jpg
Tags: horses, sea, horizontal, nature
----------------------------------------
Name: Multi Waves Epoxy Resin Wall Clock
Category: wall_clock
Image: products/clock002.jpg
Tags: resin, waves, multi-color, round, epoxy
----------------------------------------
Name: Green Beach View Epoxy Resin Wall Clock
Category: wall_clock
Image: products/clock001.jpg
Tags: resin, green, beach, round, epoxy
----------------------------------------
Name: Arabic Calligraphy Premium Acrylic Vertical Wall Art
Category: acrylic
Image: products/acrylic006.jpg
Tags: calligraphy, arabic, vertical, 